<a href="https://colab.research.google.com/github/JonathanSum/Pytorch_Lighting_example/blob/master/Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [4]:
x = np.array([[1,2,3],[4,5,6],[7,8,9]]); print(x);

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [5]:
# a1, a2 = np.split(x,1,axis=-1)

In [6]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import random_split, DataLoader


In [7]:
model = nn.Sequential(
    nn.Linear(28*28, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

In [8]:
optimiser = optim.SGD(model.parameters(), lr=1e-2)
loss = nn.CrossEntropyLoss()

In [9]:
train_data = datasets.MNIST("data", train = True, download = True, transform = transforms.ToTensor())
print(len(train_data))
train, val = random_split(train_data, [55000, 5000])
train_loader = DataLoader(train, batch_size = 32)
val_loader = DataLoader(val, batch_size = 32)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
60000


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
nb_epochs = 5
for epoch in range(nb_epochs):
  losses = list()
  accuracies = list()
  for batch in train_loader:
    #x: b x 1x 28 x 28
    x, y = batch
    b = x.size(0)
    x = x.view(b, -1)       #send x to cuda after creating the Resnet because it was definded on CPU

    # 1 forward
    l = model(x)  #l: logits

    # 2
    J = loss(l, y)           #y here too

    # 3 cleaning the gradients
    model.zero_grad()
    # optimiser.zero_grad()
    # params.grad._zero()

    # 4 accumulate the partial derivative of J with respect to param
    J.backward()


    # 5 step in the opposite direction of the gradient
    optimiser.step()
    # with torch.no_grad(): params = params - eta * params.grad
    losses.append(J.item())
    # accuracies.append(y.eq(l.detch().argmax(dim=1)).float().mean())
    #accuracies.append(y.eq(l.detch().argmax(dim=1).cpu()).float().mean())
    # accuracies.append(y.eq(l.detch().argmax(dim=1)).float().mean())

  print(f'Epoch {epoch + 1}, train loss: {torch.tensor(losses).mean():.2f}')

# model.eval()

  losses = list()
  # for epoch in range(nb_epochs):

  # accuracies = list()
  for batch in val_loader:
    #x: b x 1x 28 x 28
    x, y = batch
    b = x.size(0)
    x = x.view(b, -1)

    # 1 forward 
    with torch.no_grad():
        l = model(x)  #l: logits

    # 2
    J = loss(l, y)

    losses.append(J.item())
    accuracies.append(y.eq(l.argmax(dim=1).cpu()).float().mean())

  print(f'Epoch {epoch + 1}, validation loss: {torch.tensor(losses).mean():.2f}')
  print(f'Epoch {epoch + 1}, validation accuracies: {torch.tensor(accuracies).mean():.2f}')


Epoch 1, train loss: 1.21
Epoch 1, validation loss: 0.50
Epoch 1, validation accuracies: 0.86
Epoch 2, train loss: 0.40
Epoch 2, validation loss: 0.36
Epoch 2, validation accuracies: 0.90
Epoch 3, train loss: 0.32
Epoch 3, validation loss: 0.32
Epoch 3, validation accuracies: 0.91
Epoch 4, train loss: 0.29
Epoch 4, validation loss: 0.29
Epoch 4, validation accuracies: 0.92
Epoch 5, train loss: 0.26
Epoch 5, validation loss: 0.27
Epoch 5, validation accuracies: 0.92


In [ ]:
# nn.Linear(28*28, 64),
# nn.ReLU(),
# nn.Linear(64, 64),
# nn.ReLU(),
# nn.Linear(64, 10)


class ResNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28*28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)
  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    # print("Debugging: ",h1.shape, " ",h2.shape)
    do = self.do(h2 + h1)
    logits = self.l3(do)
    return logits
model = ResNet()

In [22]:
params = model.parameters()
optimiser = optim.SGD(model.parameters(), lr=1e-2)
loss = nn.CrossEntropyLoss()

In [24]:
nb_epochs = 5
for epoch in range(nb_epochs):
  losses = list()
  accuracies = list()
  model.train()
  for batch in train_loader:
    #x: b x 1x 28 x 28
    x, y = batch
    b = x.size(0)
    # print("Debugging  x.size(0): ", x.size(0))
    # print("Debugging  x.view(b, -1): ",x.view(b, -1).shape)
    x = x.view(b, -1)       #send x to cuda after creating the Resnet because it was definded on CPU

    # 1 forward
    l = model(x)  #l: logits
    # import pdb; pdb.set_trace()


    # 2
    J = loss(l, y)           #y here too

    # 3 cleaning the gradients
    model.zero_grad()
    # optimiser.zero_grad()
    # params.grad._zero()

    # 4 accumulate the partial derivative of J with respect to param
    J.backward()


    # 5 step in the opposite direction of the gradient
    optimiser.step()
    # with torch.no_grad(): params = params - eta * params.grad
    losses.append(J.item())
    # accuracies.append(y.eq(l.detch().argmax(dim=1)).float().mean())

    #meat part
    # accuracies.append(y.eq(l.argmax(dim=1).cpu()).float().mean())
    accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())

    # accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())

  print(f'Epoch {epoch + 1}, train loss: {torch.tensor(losses).mean():.2f}', end=',')
  print(f'Epoch {epoch + 1}, train accuracies: {torch.tensor(accuracies).mean():.2f}')



  losses = list()
  accuracies = list()
  model.eval()
  # for epoch in range(nb_epochs):


  for batch in val_loader:
    #x: b x 1x 28 x 28
    x, y = batch
    b = x.size(0)
    x = x.view(b, -1)

    # 1 forward   
    with torch.no_grad():
        l = model(x)  #l: logits

    # 2
    J = loss(l, y)

    losses.append(J.item())

    #meat part
    # accuracies.append(y.eq(l.argmax(dim=1).cpu()).float().mean())
    accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())

  # print(f'Epoch {epoch + 1}', end=',')
  # print(f'validation loss: {torch.tensor(losses).mean():.2f}', end=',')
  # print(f'validation loss: {torch.tensor(losses).mean():.2f}', end=',')
  print(f'AfterEpoch {epoch + 1}, validation loss: {torch.tensor(losses).mean():.2f}', end=',')
  print(f'After Epoch {epoch + 1}, validation accuracies: {torch.tensor(accuracies).mean():.2f}')

Epoch 1, train loss: 0.22,Epoch 1, train accuracies: 0.93
AfterEpoch 1, validation loss: 0.21,After Epoch 1, validation accuracies: 0.94
Epoch 2, train loss: 0.21,Epoch 2, train accuracies: 0.94
AfterEpoch 2, validation loss: 0.20,After Epoch 2, validation accuracies: 0.94
Epoch 3, train loss: 0.19,Epoch 3, train accuracies: 0.95
AfterEpoch 3, validation loss: 0.19,After Epoch 3, validation accuracies: 0.95
Epoch 4, train loss: 0.17,Epoch 4, train accuracies: 0.95
AfterEpoch 4, validation loss: 0.17,After Epoch 4, validation accuracies: 0.95
Epoch 5, train loss: 0.16,Epoch 5, train accuracies: 0.95
AfterEpoch 5, validation loss: 0.16,After Epoch 5, validation accuracies: 0.95


In [17]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 10)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1))
        do = self.do(h2 + h1)
        logits = self.l3(do)
        return logits
model = ResNet()


In [ ]:
class ResNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28*28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)
  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    # print("Debugging: ",h1.shape, " ",h2.shape)
    do = self.do(h2 + h1)
    logits = self.l3(do)
    return logits
model = ResNet()